# HOML CHAPTER 2 NOTES
# ~~ ML PROJECT FUNDEMENTALS ~~

The general steps for any ML project are:  
1. Understand the Problem, outcome and big picture  
2. Get the data & create a test-set   
3. Explore the data  
4. Transform and train the model(s)  
5. Evaluate the model, pivot if necessary  
6. Fine-tune the model  
7. Test the model.
- If satisfactory, prepare to launch + maintain  
- Else, goto 5.  


## Key concepts  

### How to sample/split data?  
- Random sampling
- Stratified sampling

### How to develop data?  
- Feature extraction / Developing attributes
- Correlation & Corr-Matrix
- Quantifying Text Data -> ordinal & one-hot encoding

### How to be DRY?  
- Pipelines approach for processing data

### How to clean data?  
- Drop data OR Drop attribute
- Fill-in the blanks (Imputer, etc)

### How to train?  
- sklearn framework: `.fit()/.fit_transform()`

### How to evaluate model(s)?
- Evaluation-> compare predictions to actual labels
- Choose a metric to quantify error (loss/utility)
    - (RMSE in this context)
- Cross-Validate / Validate  
    - Test model with trg data split into smaller groups (K-Fold)  

### How to finalise model(s)?  
- Fine-tune hyperparams (with training data)  
    - (Grid search, ensemble...)  
- **Test-set** the model and see Generalisation Error as the final dev step


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Get the Data

This is 'step 2' of the ML project process.

The housing data is stored locally under ./datasets/housing/housing.csv

Backend processing has been done to the raw .tgz file to convert it to the csv. Reflected under the housing.py script


In [ ]:
HOUSING_PATH = ".\datasets\housing"

def load_housing_data(housing_path=HOUSING_PATH)->pd.DataFrame:
    csv_path = os.path.join(housing_path, "housing.csv")
    print(csv_path)
    return pd.read_csv(csv_path)

### Visualising the data
- Plotting histogram
- Printing the dataframe description & info

NOTE: Some data parameters look weird as there were value caps & processing put in place

In [ ]:
housingdf = load_housing_data(HOUSING_PATH)
housingdf.info()
#housingdf.describe()

In [ ]:
housingdf.hist(bins=50, figsize=(20,15))
plt.show()

## Creating the Test Set  

### RANDOM SAMPLING  

### SHUFFLING AND PULLING DATA  

NOTE:
- When shuffling indices, be sure to set a constant seed
- This prevents repeated tests from being able to eventually see the entire dataset
- iloc refers to index location. Returns the entire row corresponding to the index

In [ ]:
def split_train_set_naiive(data, test_ratio):
    np.random.seed(42)
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(test_ratio*len(data))
    train_indices = shuffled_indices[test_set_size:]
    test_indices = shuffled_indices[:test_set_size]
    # Seperate the data
    test_set = data.iloc[test_indices]
    train_set = data.iloc[train_indices]
    return (test_set, train_set)

test_set, train_set = split_train_set_naiive(housingdf, 0.2)

### CONSISTENT RANDOMIZATION: HASHING AND UPDATING DATASET

``` split_train_set_naiive ``` is naiive because it breaks when the dataset is updated.  

A possible remedy suggested is to hash each data instances' identifier, +/- 20% max hash value.

```test_train_check``` explanation:
- `& 0xffffffff` is a mask to ensure py2 & py3 compatability
- crc32 computes a has integer value `n` such that 0<n<pow(2,32)
- A data instance is selected when `n` falls in the `test_ratio` range (test_ratio*MAX_VAL) where MAX_VAL == pow(2,32)
- `data.loc` returns data based on bools 

In [ ]:
# zlib is a data compression library
# crc32 is a checksum x32bit algorithm
from zlib import crc32

# Mark data instances where it is hashed to be part of the test set
def test_set_check(indentifier, test_ratio):
    return crc32(np.int64(indentifier)) & 0xffffffff < test_ratio * 2**32

def split_df_by_id(data:pd.DataFrame, test_ratio, id_column):
    ids = data[id_column]
    # is_in_test_set is a series of bools
    is_in_test_set = ids.apply(lambda id: test_set_check(id, test_ratio))
    # return test_set, train_set
    return data.loc[is_in_test_set], data.loc[~is_in_test_set]

# Give each data vector an identifier based on index
# Reset the '#' index to get a proper labelled index column
housingdf_with_id = housingdf.reset_index() # adds index column
test_set, train_set = split_df_by_id(housingdf_with_id, 0.2, "index")

# Alternatively, use a 'stable' param as the unique identifier
housingdf_with_id["uniq_id"] = housingdf["longitude"]*1000 + housingdf["latitude"]
test_set, train_set = split_df_by_id(housingdf_with_id, 0.2, "uniq_id")

### USE SCIKIT-LEARN TO DO DATA SPLITTING
Useful utility baked in.

In [ ]:
from sklearn.model_selection import train_test_split

rand_train_set, rand_test_set = train_test_split(housingdf, test_size=0.2, random_state=42)

### STRATIFIED SAMPLING

1. Pure random sampling can lead to bias. https://stats.stackexchange.com/questions/294151/could-someone-explain-how-this-estimate-number-is-being-arrived-at  
2. Hence, we use stratified sampling to get a more **representative** view of the data.
3. The following code stratifies `housingdf["median_income"]` into category buckets

In [ ]:
housingdf["income_cat"] = pd.cut(housingdf["median_income"], bins=[0.,1.5,3.0,4.5,6.,np.inf], labels=[1,2,3,4,5])
housingdf["income_cat"].hist()
plt.show()

# Utility to ensure cells can be run in order
rand_train_set, rand_test_set = train_test_split(housingdf, test_size=0.2, random_state=42)
train_set, test_set = train_test_split(housingdf, test_size=0.2, random_state=42)


### STRATIFIED SAMPLING WITH SCIKIT-LEARN  
Awesome library.

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in sss.split(housingdf, housingdf["income_cat"]):
    strat_train_set = housingdf.loc[train_index]
    strat_test_set = housingdf.loc[test_index]

In [ ]:
# Check that the proportion of data in the buckets are similar
_a = strat_train_set["income_cat"].value_counts() / len(strat_train_set)
_b = housingdf["income_cat"].value_counts() / len(housingdf)
print(_a-_b) #lower the value, the more similar the proportion

### SHOWING THE EFFECT OF RANDOM SAMPLING BIAS

1. The cell below takes in the randomly sampled `test_set`, stratified `strat_test_set` and full `housingdf`.  
2. It computes the proportion and shows the % deviation from actual.  
3. Stratified sampling is better than Random sampling.

In [ ]:
def compute_proportions(data:pd.DataFrame):
    return data["income_cat"].value_counts() / len(data)
rand_prop = compute_proportions(rand_test_set)
strat_prop = compute_proportions(strat_test_set)
actual_prop = compute_proportions(housingdf)
compare_prop_df = pd.DataFrame({
    "Actual": actual_prop,
    "Random": rand_prop,
    "Stratified": strat_prop
    })
compare_prop_df["Rand Err %"] = 100*compare_prop_df["Random"]/compare_prop_df["Actual"] - 100
compare_prop_df["Strat Err %"] = 100*compare_prop_df["Stratified"]/compare_prop_df["Actual"] -100
compare_prop_df

In [ ]:
# Removing income cat to follow book
for col in  (strat_test_set, strat_train_set):
    col.drop("income_cat", axis=1, inplace=True)
strat_test_set

## VISUALIZING DATA FOR INSIGHTS
- Map emulation using LONG-LAT scatter
- Multi-variable, multi-color scatter plot
- Calculating Std. Correlation Coefft (r)
    - Effective for finding LINEAR correlations
- Scatter matrix
- Developing attributes to get insights

In [ ]:
# Make a copy of the data
housing = strat_train_set.copy()

# Map plot
housing.plot(kind='scatter', x='longitude', y='latitude', alpha=0.2, figsize=(10,7))

In [ ]:
housing.plot(kind='scatter',x='longitude',y='latitude', alpha=0.3,
                s=housing["population"]/100, label="population", figsize=(10,7),
                c='median_house_value', cmap=plt.get_cmap("jet"), colorbar=True,
)
plt.legend()

In [ ]:
# Standard correlation coefficient, r: -1<r<1
# pd.DF.corr() -> computes r btwn every pair of attributes
corr_matrix = housing.corr()
corr_matrix['median_house_value'].sort_values(ascending=False)

In [ ]:
# Pandas scatter matrix compares every numeric attr with the other numeric attrs
from pandas.plotting import scatter_matrix
attribs = ["median_house_value", "median_income", "total_rooms", "housing_median_age"]
scatter_matrix(housing[attribs], figsize=(12,12))

In [ ]:
housing.plot(kind="scatter", x="median_income",y="median_house_value", alpha=0.1, figsize=(6,6))

In [ ]:
housing["rooms_per_household"] = housing["total_rooms"]/housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"]/housing["total_rooms"]
housing["population_per_household"] = housing["population"]/housing["households"]
corr_matrix = housing.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)
# Observe that the developed feature, 'bedrooms_per_room' has a stronger correlation than 'rooms' or 'bedrooms'

## Preparing the Data for ML

- Build data transformation functions

### DATA CLEANING

### DEALING WITH MISSING DATA
The dataset needs to have complete data
For missing values, there are some remedies:  

1. **Remove the affected datapoints** (districts in this context)  
- pd.DataFrame.dropna()
2. **Remove the attribute/variable**  
- pd.DataFrame.drop()
3. **Set the values to some value** (0, mean, median)  
- pd.DataFrame.fillna()

### DEALING WITH TEXT/CAT DATA
1. Convert the text data into numbers based on category
- WARNING: similar attributes may be treated as far apart if their indexes are far apart
2. Convert the data into **one-hot** encoding (0-1)


In [ ]:
# Revert data back to normal for exercise purposes.
# NOTE: .drop() returns a copy. The data src is NOT modified
# Remove the predictors and target values
housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

""" EXAMPLE CODE ON HANDLING MISSING DATA
housing.dropna(subset=["total_bedrooms"])
housing.drop("total_bedrooms", axis=1)
median = housing["total_bedrooms"].median()
housing["total_bedrooms"].fillna(median, inplace=True)
"""
# Use Scikit-learn instead
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")
# Drop non-numeric attribute. imputer cannot work for text
housing_num = housing.drop("ocean_proximity", axis=1)
imputer.fit(housing_num) # imputer calculated the median of every attribute, storing it under statistics_
transformed = imputer.transform(housing_num)
housing_tr = pd.DataFrame(transformed, columns=housing_num.columns, index = housing_num.index)

In [ ]:
# Dealing with text/categorical data
from sklearn.preprocessing import OrdinalEncoder
housing_cat = housing[["ocean_proximity"]] # Double square brace to force return dataframe
ord_encoder = OrdinalEncoder()
housing_cat_encoded = ord_encoder.fit_transform(housing_cat)
ord_encoder.categories_

In [ ]:
# Alternatively, use 1hot vector representation to indicate near sea or not
from sklearn.preprocessing import OneHotEncoder
catEncoder = OneHotEncoder()
housing_cat_1hot = catEncoder.fit_transform(housing_cat)
housing_cat_1hot

In [ ]:
# Create transformation classes to keep things DRY
from sklearn.base import BaseEstimator, TransformerMixin
rooms_ix, bedrooms_ix, population_ix, households_ix = 3,4,5,6

class CombinedAttrAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room=True):
        self.add_bedrooms_per_room=add_bedrooms_per_room
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        # [:, n] -> ignore all the rows except the nth column
        # np.c_ -> concatenate slices to the second axis
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]
    #endclass
attr_adder = CombinedAttrAdder(add_bedrooms_per_room=False)
housing_extra_attrs = attr_adder.transform(housing.values)


## PIPELINES -> MAKING TRANSFORMATIONS SEQUENTIAL

When there are many transformations to the data that needs to be done, `pipelines` (sklearn.Class) can be used to organise that process.

NUMERICAL PIPELINE 
- `Pipeline` takes in a list of name/estimator pairs
- All except the last estimator MUST be transformers (have `.fit_transform()`)
- `pipeline.fit()` calls `fir_transform()` for all transformers, and `fit()` for the last estimator

COLUMN TRANSFORMER
- `ColumnTransformer` takes in a list of tuples. The tuples contain a name, transformer and list(names)/indices/columns to be transformed
- Returns a sparse or dense matrix based on density estimation (?)

In [ ]:
# SCIKIT Pipelines to automate and sequence transformation steps
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('attribs_adder', CombinedAttrAdder()),
    ('std_scaler', StandardScaler()),
])
housing_num_tr = num_pipeline.fit_transform(housing_num)

In [ ]:
# Handling CATEGORICAL and NUMERIC conversions in the same pipeline
from sklearn.compose import ColumnTransformer

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]
full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', OneHotEncoder(), cat_attribs),
])
housing_prepared  = full_pipeline.fit_transform(housing)

## SELECT AND TRAIN THE MODEL

### TRAINING AND EVALUATING TRG SET
NOTE: This exercise is a linear regression

In [ ]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
# Fit the model based on the prepared data. This instantiates lin_reg.
lin_reg.fit(housing_prepared, housing_labels)

In [ ]:
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)
print("Predictions: ", lin_reg.predict(some_data_prepared))
print("Labels: ", list(some_labels))

In [ ]:
## Check loss (RMSE)
from sklearn.metrics import mean_squared_error
housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse
# RMSE is quite high. Model is likely underfitting data.

In [ ]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)

housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse
# RMSE is too low. Model may have overfit data.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()
forest_reg.fit(housing_prepared, housing_labels)
housing_predictions = forest_reg.predict(housing_prepared)
forest_rmse = np.sqrt(mean_squared_error(housing_labels, housing_predictions))
forest_rmse

### CROSS-VALIDATION
Split the training set into many training sets and score the predictions

In [ ]:
# Split the training set into train & dev set
# Can use K-fold cross-validation
from sklearn.model_selection import cross_val_score

def show_scores(scores):
    print("Scores: ", scores)
    print("Mean: ", scores.mean())
    print("Std Deviation: ", scores.std())

tree_scores = cross_val_score(tree_reg, housing_prepared, housing_labels, scoring='neg_mean_squared_error', cv=10)
tree_rmse_scores = np.sqrt(-tree_scores)

lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels, scoring='neg_mean_squared_error', cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
forest_scores = cross_val_score(forest_reg, housing_prepared, housing_labels, scoring='neg_mean_squared_error', cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)

show_scores(tree_rmse_scores)
show_scores(lin_rmse_scores)
show_scores(forest_rmse_scores)



## Fine-Tuning the Model

### GRIDSEARCH (CROSS-VAL)

### RANDOMISED SEARCH (covered later on)

### ENSEMBLE (covered later on)

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = [
    {'n_estimators':[3,10,30], 'max_features':[2,4,6,8]},
    {'bootstrap':[False],'n_estimators':[3,10],'max_features':[2,3,4]},
]
forest_reg = RandomForestRegressor()
gridsearch = GridSearchCV(forest_reg, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
gridsearch.fit(housing_prepared, housing_labels)

# Once all the permutations have veen trained, get the best hyper parameters
gridsearch.best_params_
gridsearch.best_estimator_
# See all the results
cvres = gridsearch.cv_results_



In [ ]:
# Analysing the best model

# See which features help with accuracy
feature_importances = gridsearch.best_estimator_.feature_importances_
extra_attribs = ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_room"]
cat_encoder = full_pipeline.named_transformers_["cat"]
car_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attribs + extra_attribs + car_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)


## TEST SET TWEAKING

### Choose the best model

In [ ]:
final_model = gridsearch.best_estimator_
x_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

x_test_prepared = full_pipeline.transform(x_test)
final_predictions = final_model.predict(x_test_prepared)
final_rmse = np.sqrt(mean_squared_error(y_test, final_predictions))
print(final_rmse)

# Check confidence interval
from scipy import stats
confidence = 0.95
squared_errors = (final_predictions - y_test) **2
np.sqrt(stats.t.interval(confidence, len(squared_errors)-1, loc=squared_errors.mean(), scale=stats.sem(squared_errors)))
